In [ ]:
import boto3, botocore
from botocore.exceptions import ClientError
import os, time, json
from datetime import date

from misc import load_from_yaml, save_to_yaml
import iam, s3, lf, rds, redshift

from misc import load_from_yaml, save_to_yaml
CONFIG_PATH = 'service_resources.yml'
CONFIG = load_from_yaml(CONFIG_PATH)

In [ ]:
account_id = os.environ['AWS_ACCOUNT_ID_ROOT']
region = os.getenv('AWS_DEFAULT_REGION')
# boto3.setup_default_session(profile_name="AMominNJ")

In [ ]:
iam_client           = boto3.client('iam')
lambda_client = boto3.client('lambda')
sns_client = boto3.client('sns')
sqs_client = boto3.client('sqs')
stepfunctions_client = boto3.client('stepfunctions')
dynamodb_client = boto3.client('dynamodb')


### IAM: Create IAM Roles and Policies

In [ ]:
iam_client.attach_role_policy(
    RoleName=role_name,
    PolicyArn=policy_arn
)

In [ ]:
lfn_role_name = 'LambdaExecutionRole'
sfn_role_name = 'StepFunctionsExecutionRole'

lambda_trust_policy = {
    "Version": "2012-10-17",
    "Statement": [
        {
            "Effect": "Allow",
            "Principal": {
                "Service": "lambda.amazonaws.com"
            },
            "Action": "sts:AssumeRole"
        }
    ]
}

stepfunctions_trust_policy = {
    "Version": "2012-10-17",
    "Statement": [
        {
            "Effect": "Allow",
            "Principal": {
                "Service": "states.amazonaws.com"
            },
            "Action": "sts:AssumeRole"
        }
    ]
}

lfn_role = iam_client.create_role(
    RoleName=lfn_role_name,
    AssumeRolePolicyDocument=json.dumps(lambda_trust_policy),
    Description=''
)

sfn_role = iam_client.create_role(
    RoleName=step_function_role_name,
    AssumeRolePolicyDocument=json.dumps(stepfunctions_trust_policy),
    Description=''
)


lambda_policy_arn = 'arn:aws:iam::aws:policy/service-role/AWSLambdaBasicExecutionRole'
dynamodb_policy_arn = "arn:aws:iam::aws:policy/AmazonDynamoDBFullAccess"
sns_policy_arn = "arn:aws:iam::aws:policy/AmazonSNSFullAccess"
sqs_policy_arn = "arn:aws:iam::aws:policy/AmazonSQSFullAccess"
states_policy_arn = "arn:aws:iam::aws:policy/AWSStepFunctionsFullAccess"


iam_client.attach_role_policy(
    RoleName=lfn_role['Role']['Arn'],
    PolicyArn=lambda_policy_arn
)
iam_client.attach_role_policy(
    RoleName=lfn_role['Role']['Arn'],
    PolicyArn=dynamodb_policy_arn
)
iam_client.attach_role_policy(
    RoleName=lfn_role['Role']['Arn'],
    PolicyArn=sns_policy_arn
)
iam_client.attach_role_policy(
    RoleName=lfn_role['Role']['Arn'],
    PolicyArn=sqs_policy_arn
)

CONFIG['iam']['lambda_role_arn'] = lfn_role['Role']['Arn']
CONFIG['iam']['lambda_role_name'] = lfn_role_name
CONFIG['iam']['step_function_role_arn'] = sfn_role['Role']['Arn']
CONFIG['iam']['step_function_role_name'] = sfn_role_name

save_to_yaml(CONFIG_PATH, CONFIG)


In [ ]:
iam.delete_iam_role(CONFIG['iam']['lambda_role_name'])
iam.delete_iam_role(CONFIG['iam']['step_function_role_name'])
iam.delete_iam_policy(CONFIG['iam']['step_function_invoke_lambda_policy'])

### Lambda Function

In [ ]:
zip_file = 'handlers.zip'
check_inventory = 'checkInventory'
calculate_total = 'calculateTotal'
redeem_points = 'redeemPoints'
bill_customer = 'billCustomer'
restore_redeem_points = 'restoreRedeemPoints'
restore_quantity = 'restoreQuantity'
sqs_worker = 'sqsWorker'

In [ ]:
lfn_check_inventory = lambda_client.create_function(
    FunctionName='checkInventory',
    Runtime='python3.9',
    Role=lfn_role['Role']['Arn'],
    Handler='handlers.check_inventory',
    Code={'ZipFile': open(zip_file, 'rb').read()},
    Timeout=300,
)
lfn_calculate_total = lambda_client.create_function(
    FunctionName='calculateTotal',
    Runtime='python3.9',
    Role=lfn_role['Role']['Arn'],
    Handler='handlers.calculate_total',
    Code={'ZipFile': open(zip_file, 'rb').read()},
    Timeout=300,
)
lfn_redeem_points = lambda_client.create_function(
    FunctionName='redeemPoints',
    Runtime='python3.9',
    Role=lfn_role['Role']['Arn'],
    Handler='handlers.redeem_points',
    Code={'ZipFile': open(zip_file, 'rb').read()},
    Timeout=300,
)
lfn_bill_customer = lambda_client.create_function(
    FunctionName='billCustomer',
    Runtime='python3.9',
    Role=lfn_role['Role']['Arn'],
    Handler='handlers.bill_customer',
    Code={'ZipFile': open(zip_file, 'rb').read()},
    Timeout=300,
)
lfn_restore_redeem_points = lambda_client.create_function(
    FunctionName='restoreRedeemPoints',
    Runtime='python3.9',
    Role=lfn_role['Role']['Arn'],
    Handler='handlers.restore_redeem_points',
    Code={'ZipFile': open(zip_file, 'rb').read()},
    Timeout=300,
)
lfn_restore_quantity = lambda_client.create_function(
    FunctionName='restoreQuantity',
    Runtime='python3.9',
    Role=lfn_role['Role']['Arn'],
    Handler='handlers.restore_quantity',
    Code={'ZipFile': open(zip_file, 'rb').read()},
    Timeout=300,
)
lfn_sqs_orker = lambda_client.create_function(
    FunctionName='sqsWorker',
    Runtime='python3.9',
    Role=lfn_role['Role']['Arn'],
    Handler='handlers.sqs_worker',
    Code={'ZipFile': open(zip_file, 'rb').read()},
    Timeout=300,
)


In [ ]:
response = lambda_client.get_function(FunctionName=check_inventory)
if response['Configuration']['State'] == 'Active':
    print(f"Lambda function {check_inventory} is active.")

- Update a Function if needed

In [ ]:
# # Check if function already exists
# lambda_client.get_function(FunctionName=check_inventory)
# # If function exists, update its code
# response = lambda_client.update_function_code(
#     FunctionName=check_inventory,
#     ZipFile=open(zip_file, 'rb').read(),
# )

In [ ]:
# lambda_client.delete_function(FunctionName=check_inventory)
# lambda_client.delete_function(FunctionName=check_inventory)

### SNS

In [ ]:
topic_name = 'NotifyCourier'
protocol = 'email'  # Change to 'sms' or 'sqs' as needed
endpoint = 'BBCRedCap3@gmail.com'  # Change to phone number or SQS queue URL as needed
message = 'This is a test message'
subject = 'Test SNS Subject'

In [ ]:
sns_attributes = {
    'DisplayName': 'My SNS Topic',
    'Policy': json.dumps({
        "Version": "2012-10-17",
        "Statement": [{
            "Effect": "Allow",
            "Principal": {"AWS": "*"},
            "Action": "SNS:Publish",
            "Resource": "*"
        }]
    }),
    'DeliveryPolicy': json.dumps({
        "http": {
            "defaultHealthyRetryPolicy": {
                "numRetries": 3,
                "numMaxDelayRetries": 0,
                "numNoDelayRetries": 0,
                "numMinDelayRetries": 1,
                "backoffFunction": "linear"
            },
            "disableSubscriptionOverrides": False
        }
    }),
    'KmsMasterKeyId': 'alias/aws/sns',
    'TracingConfig': 'Active'
}


In [ ]:
# Create the SNS topic
sns_topic = sns_client.create_topic(Name=topic_name)
# Set topic attributes
for key, value in sns_attributes.items():
    sns_client.set_topic_attributes(
        TopicArn=sns_topic['TopicArn'],
        AttributeName=key,
        AttributeValue=value
    )

In [ ]:
topic_subscription = sns_client.subscribe(
    TopicArn=sns_topic['TopicArn'],
    Protocol=protocol,
    Endpoint=endpoint
)
subscription_arn = topic_subscription['SubscriptionArn']

In [ ]:
topic_publish = sns_client.publish(
    TopicArn=sns_topic['TopicArn'],
    Message=message,
    Subject=subject
)
message_id = topic_publish['MessageId']

In [ ]:
# sns_client.delete_topic(TopicArn=sns_topic['TopicArn'])

### SQS

In [ ]:

attributes_sample = {
    'DelaySeconds': '0',
    'MaximumMessageSize': '262144',  # 256 KB
    'MessageRetentionPeriod': '345600',  # 4 days
    'ReceiveMessageWaitTimeSeconds': '0',
    'VisibilityTimeout': '30',
    'RedrivePolicy': json.dumps({
        'deadLetterTargetArn': 'arn:aws:sqs:us-east-1:123456789012:MyDeadLetterQueue',
        'maxReceiveCount': '5'
    }),
    'KmsMasterKeyId': 'alias/aws/sqs',
    'KmsDataKeyReusePeriodSeconds': '300',
    'FifoQueue': 'false',  # Set to 'true' if creating a FIFO queue
    'ContentBasedDeduplication': 'false'  # Set to 'true' if creating a FIFO queue with content-based deduplication
}

attributes={
    'DelaySeconds': '5',
    'MessageRetentionPeriod': '86400'  # 1 day
}

queue_name = 'OrdersQueue'
message_body = 'This is a SQS test message'
message_attributes = {
    'Author': {
        'StringValue': 'James',
        'DataType': 'String'
    },
    'Timestamp': {
        'StringValue': '2024-06-03T12:00:00Z',
        'DataType': 'String'
    }
}

In [ ]:
# Create the SQS queue with the specified attributes
sqs_q = sqs_client.create_queue(
    QueueName=queue_name,
    Attributes=attributes
)

In [ ]:
sent_msg = sqs_client.send_message(
    QueueUrl=sqs_q['QueueUrl'],
    MessageBody=message_body,
    MessageAttributes=message_attributes or {}
)
message_id = sent_msg['MessageId']

In [ ]:
max_number_of_messages=10, wait_time_seconds=10
received_msg = sqs_client.receive_message(
    QueueUrl=sqs_q['QueueUrl'],
    MaxNumberOfMessages=max_number_of_messages,
    WaitTimeSeconds=wait_time_seconds
)
messages = received_msg.get('Messages', [])

In [ ]:
for message in messages:
    print(f"Message: {message['Body']}")
    sqs_client.delete_message(
        QueueUrl=sqs_q['QueueUrl'],
        ReceiptHandle=message['ReceiptHandle']
    )

In [ ]:
# sqs_client.delete_queue(QueueUrl=sqs_q['QueueUrl'])

### Step Function

In [ ]:
lambda_arns = CONFIG['lambda_functions']
step_function_role_arn = CONFIG['iam']['step_function_role_arn']


# 3. Create Step Functions state machine
state_machine_definition = {
    "Comment": "Store Checkout Flow",
    "StartAt": "checkInventory",
    "States": {
        "checkInventory": {
            "Type": "Task",
            "Resource": lfn_check_inventory['FunctionArn'],
            "Catch": [
                {
                    "ErrorEquals": ["BookNotFound"],
                    "Next": "BookNotFoundError"
                },
                {
                    "ErrorEquals": ["BookOutOfStock"],
                    "Next": "BookOutOfStockError"
                }
            ],
            "ResultPath": "$.book",
            "Next": "calculateTotal"
        },
        "calculateTotal": {
            "Type": "Task",
            "Resource": lfn_calculate_total['FunctionArn'],
            "ResultPath": "$.total",
            "Next": "isRedeemNeeded"
        },
        "isRedeemNeeded": {
            "Type": "Choice",
            "Choices": [
                {
                    "Variable": "$.redeem",
                    "BooleanEquals": True,
                    "Next": "RedeemPoints"
                }
            ],
            "Default": "BillCustomer"
        },
        "RedeemPoints": {
            "Type": "Task",
            "Resource": lfn_redeem_points['FunctionArn'],
            "ResultPath": "$.total",
            "Catch": [
                {
                    "ErrorEquals": ["States.ALL"],
                    "Next": "RedeemPointsError"
                }
            ],
            "Next": "BillCustomer"
        },
        "BillCustomer": {
            "Type": "Task",
            "Resource": lfn_bill_customer['FunctionArn'],
            "ResultPath": "$.billingStatus",
            "Retry": [
                {
                    "ErrorEquals": ["States.ALL"],
                    "MaxAttempts": 0
                }
            ],
            "Catch": [
                {
                    "ErrorEquals": ["States.ALL"],
                    "ResultPath": "$.customerBilling",
                    "Next": "BillingError"
                }
            ],
            "Next": "PrepareOrder"
        },
        "PrepareOrder": {
            "Type": "Task",
            "Resource": "arn:aws:states:::sqs:sendMessage.waitForTaskToken",
            "Parameters": {
                "QueueUrl": f"https://sqs.{region}.amazonaws.com/{account_id}/OrdersQueue",
                "MessageBody": {
                    "Input.$": "$",
                    "Token.$": "$$.Task.Token"
                }
            },
            "ResultPath": "$.courierStatus",
            "Catch": [
                {
                    "ErrorEquals": ["NoCourierAvailable"],
                    "ResultPath": "$.courierError",
                    "Next": "RefundCustomer"
                }
            ],
            "Next": "DispatchOrder"
        },
        "DispatchOrder": {
            "Type": "Task",
            "Resource": "arn:aws:states:::sns:publish",
            "Parameters": {
                "TopicArn": f"arn:aws:sns:{region}:{account_id}:NotifyCourier",
                "Message.$": "$"
            },
            "Next": "Dispatched"
        },
        "Dispatched": {
            "Type": "Pass",
            "Result": "Your order will be dispatched in 24 hours",
            "End": True
        },
        "RestoreRedeemPoints": {
            "Type": "Task",
            "Resource": restore_redeem_points['FunctionArn'],
            "End": True
        },
        "RestoreQuantity": {
            "Type": "Task",
            "Resource": restore_quantity['FunctionArn'],
            "ResultPath": "$.quantityRestoreStatus",
            "Next": "RestoreRedeemPoints"
        },
        "RefundCustomer": {
            "Type": "Pass",
            "Result": "Customer is refunded",
            "ResultPath": "$.refundStatus",
            "Next": "RestoreQuantity"
        },
        "BookNotFoundError": {
            "Type": "Pass",
            "Result": "No such book available",
            "End": True
        },
        "BookOutOfStockError": {
            "Type": "Pass",
            "Result": "Sorry, the book is out of stock",
            "End": True
        },
        "RedeemPointsError": {
            "Type": "Pass",
            "Result": "Error in redeeming points",
            "End": True
        },
        "BillingError": {
            "Type": "Pass",
            "Result": "Billing error",
            "ResultPath": "$.billingStatus",
            "Next": "RestoreRedeemPoints"
        }
    }
}

state_machine_arn = stepfunctions_client.create_state_machine(
    name="storeCheckoutFlow",
    definition=json.dumps(state_machine_definition),
    roleArn=sfn_role['Role']['Arn']
)['stateMachineArn']


In [ ]:
# Construct the policy document for Step Functions
policy_document = {
    "Version": "2012-10-17",
    "Statement": [
        {
            "Effect": "Allow",
            "Action": "states:StartExecution",
            "Resource": state_machine_arn
        }
    ]
}

lambda_function_arns = [lfn_check_inventory, lfn_bill_customer, lfn_calculate_total, lfn_restore_quantity, lfn_redeem_points, lfn_restore_redeem_points]

# Add Lambda permissions to the policy document
for lambda_arn in lambda_function_arns:
    policy_statement = {
        "Effect": "Allow",
        "Action": "lambda:InvokeFunction",
        "Resource": lambda_arn
    }
    policy_document['Statement'].append(policy_statement)

# Convert policy document to JSON string
policy_json = json.dumps(policy_document)

# Create a policy

policy_creation_response = iam_client.create_policy(
    PolicyName='StepFunctionInvokeLambdaPolicy',
    PolicyDocument=policy_json,
    Description='Allows Step Functions to invoke Lambda functions'
)
# Attach the policy to the Step Functions role
iam_client.attach_role_policy(
    RoleName=sfn_role_name,
    PolicyArn=policy_creation_response['Policy']['Arn']
)


In [ ]:
# stepfunctions_client.delete_state_machine(stateMachineArn=state_machine_arn)

### Dynamo DB

In [ ]:
user_table_name = 'userTable'
book_table_name = 'bookTable'

user_key_schema = [
    {
        'AttributeName': 'userId',
        'KeyType': 'HASH'  # Partition key
    }
]

user_attribute_definitions = [
    {
        'AttributeName': 'userId',
        'AttributeType': 'S'  # String
    }
]

book_key_schema = [
    {
        'AttributeName': 'bookId',
        'KeyType': 'HASH'  # Partition key
    }
]

book_attribute_definitions = [
    {
        'AttributeName': 'bookId',
        'AttributeType': 'S'  # String
    }
]


In [ ]:

user_table = dynamodb_client.create_table(
    TableName=user_table_name,
    KeySchema=user_key_schema,
    AttributeDefinitions=user_attribute_definitions,
    ProvisionedThroughput={
        'ReadCapacityUnits': 5,
        'WriteCapacityUnits': 5
    }
)

book_table = dynamodb_client.create_table(
    TableName=book_table_name,
    KeySchema=book_key_schema,
    AttributeDefinitions=book_attribute_definitions,
    ProvisionedThroughput={
        'ReadCapacityUnits': 5,
        'WriteCapacityUnits': 5
    }
)

In [ ]:
print(f"Waiting for {book_table_name} to become active...")
waiter = dynamodb_client.get_waiter('table_exists')
waiter.wait(TableName=book_table_name)
print(f"{book_table_name} is now active.")


In [ ]:

# Put items into tables
user_item = {
    'userId': {'S': '1'},
    'name': {'S': 'James'},
    'points': {'N': '300'}
}

# Put items into table
book_item = {
    'bookId': {'S': '1'},
    'title': {'S': 'Algorithm and Data Structures'},
    'price': {'N': '19.99'},
    'quantity': {'N': '100'}
}

dynamodb_client.put_item(TableName=user_table_name,Item=user_item)
dynamodb_client.put_item(TableName=book_table_name,Item=book_item)


In [ ]:
# response = dynamodb_client.delete_table(TableName=user_table_name)
# response = dynamodb_client.delete_table(TableName=book_table_name)